In [402]:
from shapely.geometry import Polygon, Point, mapping
from shapely.ops import cascaded_union
import folium
p1 = Polygon([
      (
        -72.2779369,
        42.9272056
      ),
      (
        -72.2778833,
        42.9253673
      ),
      (
        -72.2757268,
        42.9254302
      ),
      (
        -72.2758663,
        42.9272685
      ),
      (
        -72.2779369,
        42.9272056
      )
    ])
p2 = Polygon([
      (
        -72.2751045,
        42.9279991
      ),
      (
        -72.2768211,
        42.9270328
      ),
      (
        -72.2763169,
        42.9265772
      ),
      (
        -72.2766709,
        42.9261844
      ),
      (
        -72.2758234,
        42.9255087
      ),
      (
        -72.2748256,
        42.9267971
      ),
      (
        -72.2751045,
        42.9279991
      )])
print(p1.intersects(p2))

def merge(p1, p2, intersection_with_former=True):
    if intersection_with_former:
        p2new = p2.difference(p1)
        p1new = p1
    else:
        p1new = p1.difference(p2)
        p2new = p2
    return p1new, p2new
p1new, p2new = (p1.difference(p2), p2.difference(p1))
m = folium.Map(location=[42.9271, -72.2751045],
                   tiles='OpenStreetMap', zoom_start=16)
# m.choropleth(geo_data=p1new)
# m.choropleth(geo_data=p2new)
m

True


In [403]:
x = p1.intersection(p2)
x1 = x
# buffer is because, polyfill returns hexgrids totally contained within the polygon. thus leaving some gaps
# between intersection and original polygons
x = x.buffer(0.001)
# m.choropleth(geo_data=x)
m

In [404]:
from h3 import h3
import geopandas as gpd
coordinates = []
for coordinate in mapping(x)["coordinates"][0]:
    coordinates.append([coordinate[0], coordinate[1]])
geojson_x = {
    "type": "Polygon",
    "coordinates": [coordinates]
}

In [405]:
# resolution here is critical. in this case if given <=11 or 12, the hexagon is really large compared 
# to size of polygons, thus giving weird outcomes. 

# it can be mapped to area while making the function

# for example a big polgon if run on resolution 15 would take a lot of time, because the number of grids will be
# too many. 
# at the same time, very smaller resolution would lead to less fine division of area.
hexgrids = h3.polyfill(geojson_x, 15)

In [406]:

p1new_buffer = []
p2new_buffer = []
p1new_buffer.append(p1new)
p2new_buffer.append(p2new)
for cell in hexgrids:
    bound = h3.h3_to_geo_boundary(cell)
    center = Point(h3.h3_to_geo(cell))
    poly = Polygon(bound)
    d1 = center.distance(p1new)
    d2 = center.distance(p2new)
    if d1 > d2:
        p2new_buffer.append(poly)
    else:
        p1new_buffer.append(poly)
p1new_merged = cascaded_union(p1new_buffer)
p2new_merged = cascaded_union(p2new_buffer)
    


In [407]:
# m.choropleth(geo_data=p1new_merged)
# m.choropleth(geo_data=p2new_merged)
# m

In [408]:
# m.choropleth(geo_data=x1)
# m

In [409]:
p1p2 = [p1, p2]
p1p2 = cascaded_union(p1p2)
p1p2_extended = [p1new_merged, p2new_merged]
p1p2_extended = cascaded_union(p1p2_extended)
extra_poly = p1p2_extended.difference(p1p2)
p1new = p1new_merged.difference(extra_poly)
p2new = p2new_merged.difference(extra_poly)

In [410]:
m.choropleth(geo_data=p1new)
m.choropleth(geo_data=p2new)
m